<p style="font-size: 100px; text-align: center; color: rgb(212, 69, 0)"> <strong>S</strong>tartup <strong>A</strong>nalysis </p>

<p style="font-size: 60px; text-align: center; color: rgb(62, 61, 60)"> <strong>P</strong>art <strong>O</strong>ne</p>

<p style="text-align: center"><img src="https://i.vimeocdn.com/portrait/4910448_300x300" style="height: 50%; text-align: center"/></p>

##### This project was conducted by Sascha Hagedorn, Maximilian Ott and Priya Matnani as part of the lecture IST 718 Advanced Information Analytics taught  by Daniel Acuna at School of Information Studies, Syracuse University. #####

# Project Abstract

The data was extracted from Crunchbase on February 2014. The dataset provides information about startup companies, investment, and acquisitions via Crunchbase.

Multiple supervised learning algorithms such as Logistic Regression, Random Forest and Neural Networks are applied after intense data preparation. Validation shows that Neural Networks has the best performance in this case.

With the results of this project existing startups can evaluate their performance in order to discover their probability to be acquired and emerging startups can use the outcomes as a guideline for how to structure their company or which features to emphasize while going down the path of an emerging startup.

The data can be retrieved from: https://data.world/datanerd/startup-venture-funding

# Imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load functionality to manipulate dataframes
from pyspark.sql import functions as fn
from pyspark.sql.functions import col, monotonically_increasing_id, split, when

# Functionality for computing features
from pyspark.ml import feature, regression, classification, Pipeline, clustering
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.feature import RFormula, Tokenizer, VectorAssembler, HashingTF, Word2Vec

from itertools import chain
from pyspark.ml.linalg import Vectors, VectorUDT

#Evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Definition of custom functions #

-- In this section custom functions are defined. This part is at the top to ensure, that "Run All" works properly.  --

Custom functions are user defined functions. We defined them in order to do certain calculations for us. Each headline indicates to which procedure the functions belong.

### functions for customized categories

In [13]:
#flatten list spark df to spark df
def flatten_list(df):
  test_pd = df.toPandas()
  mylist1 = []
  klist = []
  for i in range(len(test_pd.kmeans_feat)):
    newlist = list(chain.from_iterable(test_pd.category[i]))
    mylist1.append(newlist)
    klist.append(i)
  transferdf_pd = pd.DataFrame({"kmeans_feat": klist, "category": mylist1})
  newcatewoo = spark.createDataFrame(transferdf_pd)
  return newcatewoo

In [14]:
def three_highest_catpluscount(df):  
  #to pandas
  test1 = df.select("tf").toPandas()
  maxi = []
  maxv = [0,0,0]
  indexlist = []
  valuelist = []
  kmeanlist = []
  for a in range(len(test1.tf)):
    for i in range(len(test1.tf[a])):
      if (min(maxv) < test1.tf[a][i]):
        maxv[0] = test1.tf[a][i]
        maxv = sorted(maxv)
    testarr = test1.tf[a].toArray()
    for n in range(2, -1, -1):
      index = np.where(testarr == maxv[n])
      maxi.append(index[0][0])
      valuelist.append(maxv[n])
      kmeanlist.append(a)
    maxv = [0,0,0]
  kmeancate_pd = pd.DataFrame({"categoryindex": maxi, "kmean_feat": kmeanlist, "count": valuelist})
  kmeancate = spark.createDataFrame(kmeancate_pd)
  return kmeancate

In [15]:
# add categories to index
def add_category_words(df):
  kmeancate_pd = df.toPandas()
  mylisti = []
  for t in range(len(kmeancate_pd)):
    ind = kmeancate_pd.categoryindex[t]
    mylisti.append(wordvector_catwoo[ind])
  kmeancate_pd["category"] = mylisti
  kmeancate_words = spark.createDataFrame(kmeancate_pd)
  return kmeancate_words

In [16]:
def create_final_category(df):
  kmeancate_pd = df.toPandas()
  multiplecate = []
  categorylist = []
  kmeanlist = []
  for o in range(0,len(kmeancate_pd),3):
    multiplecate.append(kmeancate_pd["category"][o])
    if((kmeancate_pd["count"][o]*0.7 <= kmeancate_pd["count"][o+1]) and (kmeancate_pd["category"][o] != kmeancate_pd["category"][o+1])):
      multiplecate.append(kmeancate_pd["category"][o+1])
      if((kmeancate_pd["count"][o+1]*0.7 <= kmeancate_pd["count"][o+2]) and (kmeancate_pd["category"][o+1] != kmeancate_pd["category"][o+2])):
        multiplecate.append(kmeancate_pd["category"][o+2])
    categorystring = " + ".join(multiplecate)
    categorylist.append(categorystring)
    kmeanlist.append(o/3)
    multiplecate = []
  final_pd = pd.DataFrame({"category_final": categorylist, "kmean_feat": kmeanlist})
  final = spark.createDataFrame(final_pd)
  return final

### functions to get majority of investor country codes and funding round types

In [18]:
def blank_as_null(x):
    return when(col(x) != "", col(x)).otherwise(None)

In [19]:
def two_highest_catpluscount(df):  
  #to pandas
  test1 = df.toPandas()
  maxi = []
  maxv = [0]
  indexlist = []
  valuelist = []
  kmeanlist = []
  for a in range(len(test1.tf)):
    for i in range(len(test1.tf[a])):
      if (min(maxv) < test1.tf[a][i]):
        maxv[0] = test1.tf[a][i]
        maxv = sorted(maxv)
    testarr = test1.tf[a].toArray()
    for n in range(1):
      index = np.where(testarr == maxv[n])
      maxi.append(index[0][0])
      valuelist.append(maxv[n])
      kmeanlist.append(test1.permalink[a])
    maxv = [0]
  kmeancate_pd = pd.DataFrame({"categoryindex": maxi, "perma": kmeanlist, "count": valuelist})
  kmeancate = spark.createDataFrame(kmeancate_pd)
  return kmeancate

In [20]:
# add categories to index
def add_inv_words(df):
  kmeancate_pd = df.toPandas()
  mylisti = []
  for t in range(len(kmeancate_pd)):
    ind = kmeancate_pd.categoryindex[t]
    mylisti.append(inv_words[ind])
  kmeancate_pd["category"] = mylisti
  kmeancate_words = spark.createDataFrame(kmeancate_pd)
  return kmeancate_words

In [21]:
# add categories to index
def add_round_words(df):
  kmeancate_pd = df.toPandas()
  mylisti = []
  for t in range(len(kmeancate_pd)):
    ind = kmeancate_pd.categoryindex[t]
    mylisti.append(round_words[ind])
  kmeancate_pd["category"] = mylisti
  kmeancate_words = spark.createDataFrame(kmeancate_pd)
  return kmeancate_words

# Get Data from Dropbox

To retrieve the data dynamically, the data is stored online at Dropbox. Thus, the path stays for all users the same and the data can be pulled easily. Github would have been a more sophistaced service for storing the data. Since the data exceeds 25MB, the files were to large for Github. The paths where the data can be found are listed below.

These paths are public and the data can be retrieved by anyone who has the link.

In [25]:
#pathnewacq = "https://www.dropbox.com/s/8xevxeuekw2mice/acquisitions.csv"
#pathnewadd = "https://www.dropbox.com/s/66g87yw6gw620y2/additions.csv"
#pathnewcom = "https://www.dropbox.com/s/d25fy3fp6bqjiid/companies.csv"
#pathnewinv = "https://www.dropbox.com/s/8cpf8osyy1hl9am/investments.csv"
#pathnewrou = "https://www.dropbox.com/s/neywvzrujmxykjr/rounds.csv"

### get files to local server

In [27]:
%sh wget https://www.dropbox.com/s/dko78rtre7job62/acquisitions1.csv -nv

In [28]:
%sh wget https://www.dropbox.com/s/66g87yw6gw620y2/additions.csv -nv

In [29]:
%sh wget https://www.dropbox.com/s/d25fy3fp6bqjiid/companies.csv -nv

In [30]:
%sh wget https://www.dropbox.com/s/8cpf8osyy1hl9am/investments.csv -nv

In [31]:
%sh wget https://www.dropbox.com/s/w3cjfl8v7cw3pcx/investments1.csv -nv

In [32]:
%sh wget https://www.dropbox.com/s/neywvzrujmxykjr/rounds.csv -nv

### load files into variables in the notebook

The data is loaded and the format .csv is specified. From now on, one can work with the data in the notebook.

In [35]:
dfacq = sqlContext.read.format("csv").load("file:///databricks/driver/acquisitions1.csv", delimiter = ",", header = True)
dfadd = sqlContext.read.format("csv").load("file:///databricks/driver/additions.csv", delimiter = ",", header = True)
dfcom = sqlContext.read.format("csv").load("file:///databricks/driver/companies.csv", delimiter = ",", header = True)
dfinv = sqlContext.read.format("csv").load("file:///databricks/driver/investments1.csv", delimiter = ",", header = True)
dfrou = sqlContext.read.format("csv").load("file:///databricks/driver/rounds.csv", delimiter = ",", header = True)

### clean errors in column names

After loading the data, we discovered that some column names have leading spaces, which leads to confusing column names since spaces in the beginning are hard to detect. This can easily cause errors and confusion. Hence, the column names are checked and the appropriate ones are renamed to a proper label. This procedure applies only to the Companies, the Acquisitions and the Rounds spreadsheets because the others did not have confusing column names.

In [38]:
dfcom = dfcom.select(col("permalink"), col("name"), col("homepage_url"), col("category_list"), col(" market ").alias("market"), col(" funding_total_usd ").alias("funding_total_usd"), col("status"), col("country_code"), col("state_code"), col("region"), col("city"), col("funding_rounds"), col("founded_at"), col("founded_month"), col("founded_quarter"), col("founded_year"), col("first_funding_at"), col("last_funding_at"))

In [39]:
dfacq = dfacq.select(col("company_permalink"), col("company_name"), col("company_category_list"), col("company_market"), col("company_country_code"), col("company_state_code"), col("company_region"), col("company_city"), col("acquirer_permalink"), col("acquirer_name"), col("acquirer_category_list"), col("acquirer_market"), col("acquirer_country_code"), col("acquirer_state_code"), col("acquirer_region"), col("acquirer_city"), col("acquired_at"), col("acquired_month"), col("acquired_quarter"), col("acquired_year"), col(" price_amount ").alias("price_amount"), col("price_currency_code"))

In [40]:
dfrou = dfrou.select(col("company_permalink"), col("company_name"), col("company_category_list"), col("company_market"), col("company_country_code"), col("company_state_code"), col("company_region"), col("company_city"), col("funding_round_permalink"), col("funding_round_type"), col("funding_round_code"), col("funded_at"), col("funded_month"), col(" funded_quarter ").alias("funded_quarter"), col("funded_year"), col(" raised_amount_usd ").alias("raised_amount_usd"))

# Data Understanding

Data undestanding deals with a rather detailed investigation of the used data. We look for null values, existing dummy values and calculate basic statistics such as mean, max or min for numerical values. Categorical features and their number of unique categories are outlined. Moreover, different features and their occurring values are counted or summarized in order to get some first insights. Our examination is clustered into the 5 different spreadsheets.

<h2> Excel Spreadsheets with column names </h2>
<br/>

<ul>
  <li>Companies</li>
  <ul>
    <li>permalink</li>
    <li>name</li>
    <li>homepage_url</li>
    <li>category_list</li>
    <li>market</li>
    <li>funding_total</li>
    <li>status</li>
    <li>country_code</li>
    <li>state_code</li>
    <li>region</li>
    <li>city</li>
    <li>funding_round</li>
    <li>founded_at</li>
    <li>founded_month</li>
    <li>founded_quarter</li>
    <li>founded_year</li>
    <li>first_funding_at</li>
    <li>last_funding_at</li>
  </ul>
  <li>Rounds</li>
  <ul>
    <li>company_permalink</li>
    <li>company_name</li>
    <li>company_category_list</li>
    <li>company_market</li>
    <li>company_country_code</li>
    <li>company_state_code</li>
    <li>company_region</li>
    <li>company_city</li>
    <li>funding_round_peramlink</li>
    <li>funding_round_type</li>
    <li>funding_round_code</li>
    <li>fundet_at</li>
    <li>funded_month</li>
    <li>funded_quarter</li>
    <li>funded_year</li>
    <li>raised_amount_usd</li>
  </ul>
    <li>Investements</li>
  <ul>
    <li>company_permalink</li>
    <li>company_name</li>
    <li>company_category_list</li>
    <li>company_market</li>
    <li>company_country_code</li>
    <li>company_state_code</li>
    <li>company_region</li>
    <li>company_city</li>
    <li>investor_permalink</li>
    <li>investor_name</li>
    <li>investor_category_list</li>
    <li>investor_market</li>
    <li>investor_country_code</li>
    <li>investor_region</li>
    <li>investor_city</li>
    <li>funding_round_permalink</li>
    <li>funding_round_type</li>
    <li>funding_round_code</li>
    <li>funded_at</li>
    <li>funded_month</li>
    <li>funded_quarter</li>
    <li>funded_year</li>
    <li>rasied_amount_us</li>
  </ul>
  <li>Acquisitions</li>
  <ul>
    <li>company_permalink</li>
    <li>company_name</li>
    <li>company_category_list</li>
    <li>company_market</li>
    <li>company_country_code</li>
    <li>company_state_code</li>
    <li>company_region</li>
    <li>company_city</li>
    <li>acquirer_permalink</li>
    <li>acquirer_name</li>
    <li>acquirer_category_list</li>
    <li>acquirer_market</li>
    <li>acquirer_country_code</li>
    <li>acquirer_state_code</li>
    <li>acquirer_region</li>
    <li>acquirer_city</li>
    <li>acquired_at</li>
    <li>acquired_month</li>
    <li>acquired_quarter</li>
    <li>acquired_year</li>
    <li>price_amount</li>
    <li>price_currency_code</li>
  </ul>
  <li>Additions</li>
  <ul>
    <li>content</li>
    <li>month_str</li>
    <li>quarter_str</li>
    <li>year_str</li>
    <li>value</li>
  </ul>

</ul>

### basic examination and statistics of companies data

In [45]:
display(dfcom)

In [46]:
dfcom.count()

In [47]:
#show distribution of target variable
display(
  dfcom.select("status").\
  groupby(col("status")).\
  agg(fn.count("status"))
)

In [48]:
#check for null values in general and after that for each category of the target variable
dfcom.toPandas().isnull().sum()

In [49]:
dfcom.filter(col("status") == "closed").toPandas().isnull().sum()

In [50]:
dfcom.filter(col("status") == "operating").toPandas().isnull().sum()

In [51]:
dfcom.filter(col("status") == "acquired").toPandas().isnull().sum()

In [52]:
#get overview of different markets and their occurrence
markets = dfcom.select("market").\
  groupby(col("market")).\
  agg(fn.count("market")).\
  sort("count(market)", ascending=False)

In [53]:
display(markets.take(10))

In [54]:
#get number of unique market categories
dfcom.select("market").distinct().count()

In [55]:
#find highest funding_total_usd and appropriate startups
display(
  dfcom.select("permalink", "name", "funding_total_usd").\
  sort("funding_total_usd", ascending=False)
)

In [56]:
#understand where most of the startups are located - get cities with startup occurrence
cities = dfcom.select("permalink", "name", "city").\
  groupBy("city").\
  agg(fn.count("city")).\
  sort("count(city)", ascending=False)

In [57]:
display(cities.take(10))

In [58]:
#understand where most of the startups are located - get countries with startup occurrence
countries = dfcom.select("permalink", "name", "country_code").\
  groupBy("country_code").\
  agg(fn.count("country_code")).\
  sort("count(country_code)", ascending=False)

In [59]:
display(countries)

In [60]:
display(
  dfcom.select("permalink", "name", "founded_year").\
  agg(fn.max("founded_year"), fn.min("founded_year"))
)

In [61]:
#get basic statistics for columns with numerical value - understand min, max, mean etc.
dfcom.select("funding_total_usd", "funding_rounds", "founded_year").describe().show()

### basic examination and statistics of acquisitions data

In [63]:
display(dfacq)

In [64]:
dfacq.count()

In [65]:
#who are the companies who acquire the most
acquirers = dfacq.\
  select("acquirer_name").\
  groupby(col("acquirer_name")).\
  agg(fn.count("acquirer_name")).\
  sort("count(acquirer_name)", ascending=False)

In [66]:
display(acquirers.take(10))

In [67]:
#statistics of price column
dfacq.select("price_amount").describe().show()

In [68]:
#which different currencies occur
display(
  dfacq.\
  select("price_currency_code").\
  groupby("price_currency_code").\
  agg(fn.count("price_currency_code")).\
  sort("count(price_currency_code)", ascending=False)
)

### basic examination and statistics of investment data

In [70]:
display(dfinv)

In [71]:
dfinv.count()

In [72]:
#check for null values
dfinv.toPandas().isnull().sum()

In [73]:
# get investor name, number of occurrences and total amount of invested usd to detect top investors
investors = dfinv.\
  select("investor_name", "raised_amount_usd").\
  groupby(col("investor_name")).\
  agg(fn.count("investor_name"), fn.sum("raised_amount_usd")).\
  sort("count(investor_name)", ascending=False)

In [74]:
display(investors.take(7))

In [75]:
#where are most of the investors located
investorcities = dfinv.\
  select("investor_city",).\
  groupby(col("investor_city")).\
  agg(fn.count("investor_city")).\
  sort("count(investor_city)", ascending=False)

In [76]:
display(investorcities.take(8))

In [77]:
#how many distinct investors do we have
dfinv.\
select("investor_permalink").\
distinct().\
count()

In [78]:
dfinv.select("raised_amount_usd", "funded_year").describe().show()

In [79]:
#how much money per year was invested over time
investperyear = dfinv.\
select("funded_year", "raised_amount_usd").\
groupby("funded_year").\
agg(fn.sum("raised_amount_usd")).\
sort("funded_year")

In [80]:
display(investperyear)

### basic examination and statistics of rounds data

In [82]:
display(dfrou)

In [83]:
dfrou.count()

In [84]:
dfrou.toPandas().isnull().sum()

In [85]:
#which funding round types happen more often than others - which are our main types
fundingrounds = dfrou.\
  select("funding_round_type").\
  groupBy("funding_round_type").\
  agg(fn.count("funding_round_type")).\
  sort("count(funding_round_type)", ascending=False)

In [86]:
display(fundingrounds.take(7))

In [87]:
dfrou.select("raised_amount_usd", "funded_year").describe().show()

# Data Preparation

We discovered that the category column contains multiple labels concatenated by a bar. This would lead to many different category combinations, even though the main category would often be the same. Therefore, we decided to first cluster our different categories. After that, the most occurring single category in each cluster is extracted and used as new representative label for this cluster. This leads to a dimensionality reduction.

## create custom category based on given category column

In [91]:
# Replace + with | , see Hardware+Software
df_categories_pd = dfcom.select("category_list").toPandas()
k = []
for i in df_categories_pd.category_list:
    i = str(i)
    j = i.replace(' + ','|')
    k.append(j)

In [92]:
# Create Spark DF with new Category list
mylist_pd_k = pd.DataFrame({"category_list": k})
df_cat_k = sqlContext.createDataFrame(mylist_pd_k)

In [93]:
#convert to Pandas
permalink_pd = dfcom.select("permalink").toPandas()

In [94]:
#create spark df of the categories as a list for each company
df_categories_pd = mylist_pd_k

mylist = []
for i in range(len(df_categories_pd)):  
  mystring = str(df_categories_pd.category_list[i])
  myremover = mystring.split("|")
  str_list = filter(None, myremover)
  mylist.append(str_list)

mylist_pd = pd.DataFrame({"category": mylist, "permalink": permalink_pd.permalink})
df_cat = sqlContext.createDataFrame(mylist_pd)

In [95]:
# counter vector + kmeans clustering + fitting and transforming
cv = feature.CountVectorizer(inputCol='category', outputCol='tf')
kmeans = clustering.KMeans(k=400, featuresCol='tf', predictionCol='kmeans_feat')
pipeline_model = Pipeline(stages=[cv, kmeans]).fit(df_cat)
df_catid_kmeans = pipeline_model.transform(df_cat)

In [96]:
pipeline_model.stages[0].vocabulary

In [97]:
#display for checking results
#display(df_catid_kmeans.filter(col("kmeans_feat") == 0))

In [98]:
#concatenate the categories which are within one kmeans cluster
df_catid_kmeans_concat = df_catid_kmeans.\
groupby("kmeans_feat").\
agg(fn.collect_list(col("category")).alias("category")).\
sort("kmeans_feat")

In [99]:
#execute custom functions before the following

#flatten list spark df to spark df
newcatewoo = flatten_list(df_catid_kmeans_concat)

In [100]:
# counter vectorizer
cv = feature.CountVectorizer(inputCol='category', outputCol='tf')

In [101]:
# cv fitting and df transformation
cv_model = cv.fit(newcatewoo)
df_catwoo_cv = cv_model.transform(newcatewoo)

In [102]:
#get the three categories occuring most often for each cluster
kmeancate = three_highest_catpluscount(df_catwoo_cv)

In [103]:
wordvector_catwoo = cv_model.vocabulary
wordvector_catwoo

In [104]:
# add categories to index
kmeancate_words = add_category_words(kmeancate)

In [105]:
#create final representative category value for each cluster
final = create_final_category(kmeancate_words)

In [106]:
display(
  final.\
  groupby("category_final").\
  agg(fn.count("category_final").alias("count")).\
  filter(col("count") > 1).\
  sort(col("count").desc())
)

In [107]:
#get number of unique categories
final.select("category_final").distinct().count()

## join spreadsheets

Since we have different spreadsheets, we decided to join spreadsheets to get one mastertable as basis for our analysis. Considering 'acquired' as target variable, only the companies and the investments spreadsheets contain significant information. Hence, those are joined. Since the investment spreadsheet can contain multiple investments for one company, the values of some features need to aggregated. While joining, we also calculated our own columns based on information of others.

### check if companies are unique

In [111]:
#look for permalinks that occur twice 
display(dfcom.select("permalink").groupBy("permalink").agg(fn.count("permalink")).where(fn.count("permalink") == 2))

In [112]:
#investigate duplicates
display(dfcom.filter((col("permalink") == "/organization/prysm")|(col("permalink") == "/organization/treasure-valley-urology-services")))

### remove duplicates with less information

In [114]:
#select observation with less information of duplicate 1
dfcomsubtr1 = dfcom.filter((col("permalink") == "/organization/prysm") & ((col("permalink") == "/organization/prysm") & (col("funding_rounds") == 1)))

In [115]:
#select observation with less information of duplicate 2
dfcomsubtr2 = dfcom.filter((col("permalink") == "/organization/treasure-valley-urology-services") & ((col("permalink") == "/organization/treasure-valley-urology-services") & (col("funding_rounds") == 1)))

In [116]:
#get rid of duplicates (deprecated entries)
dfcom = dfcom.subtract(dfcomsubtr1).subtract(dfcomsubtr2)

In [117]:
#double check if preparation worked out
display(dfcom.select("permalink").groupBy("permalink").agg(fn.count("permalink")).sort("count(permalink)", ascending=False))

### define calculations for aggregation when joining the spreadsheets

In [119]:
#calculate the age of a company based on year of founding, the appropriate quarter and 2015, which is the date when the data set was created
age_calc = fn.when(col("quarter_new") == "Q2", 2015 - col("founded_year") - 0.25).\
  otherwise(fn.when(col("quarter_new") == "Q3", 2015 - col("founded_year") - 0.5).\
            otherwise(fn.when(col("quarter_new") == "Q4", 2015 - col("founded_year") - 0.75).\
                      otherwise(2015 - col("founded_year"))))

In [120]:
#calculate the time to funding of a company based on date of funding and date of data set creation 
time_to_funding_calc = fn.when(col("funded_quarter_new") == "Q2", col("funded_year") + 0.25 - (2015 - col("age"))).\
  otherwise(fn.when(col("funded_quarter_new") == "Q3", col("funded_year") + 0.5 - (2015 - col("age"))).\
            otherwise(fn.when(col("funded_quarter_new") == "Q4", col("funded_year") + 0.75 - (2015 - col("age"))).\
                      otherwise(col("funded_year") - (2015 - col("age")))))

### define subsets before joining

In [122]:
#select subsets before joining to exclude information/observations which will not be used
dfcomsub = dfcom.\
  withColumn("quarter_new", col("founded_quarter").substr(6,2)).\
  withColumn("age", age_calc).\
  select("permalink", "name", "market", "funding_total_usd", "status", "country_code", "city", "funding_rounds", "founded_year", "quarter_new","age")

dfinvsub = dfinv.\
  withColumn("funded_quarter_new", col("funded_quarter").substr(6,2)).\
  select("company_permalink", "investor_permalink", "investor_name", "investor_country_code", "funding_round_type", "funded_quarter_new", "funded_year", "raised_amount_usd")

### join companies and investments data

In [124]:
#multiple rows in dfinv for each permalink
dfmaster = dfcomsub.join(dfinvsub, dfcomsub["permalink"] == dfinvsub["company_permalink"], 'leftouter')
dfmaster2 = dfmaster.\
  withColumn("time_to_funding", time_to_funding_calc)

### aggregate multiple entries for each company

In [126]:
dfmaster2_agg = dfmaster2.\
  groupby(col("permalink").alias("permalink_agg")).\
  agg(fn.count("investor_permalink").alias("count_investor"),
      fn.min("time_to_funding").alias("time_to_first_funding"),
      fn.concat_ws(", ", fn.collect_list(col("investor_country_code"))).alias("investor_country_codes"),
      fn.concat_ws(", ", fn.collect_list(col("funding_round_type"))).alias("funding_round_types"),
      fn.sum("raised_amount_usd").alias("total_raised_usd"))
     


In [127]:
dfmaster2_agg.count()

### join aggregated values to companies subset dataframe

In [129]:
dfmaster_final = dfcomsub.join(dfmaster2_agg, dfcomsub["permalink"] == dfmaster2_agg["permalink_agg"], 'leftouter')

In [130]:
display(dfmaster_final)

In [131]:
dfmaster_final.count()

In [132]:
#examination of observations with target variable acquired
display(dfmaster.where(col("status") == "acquired"))

In [133]:
#drop column which exists twice with different name
dfmaster3 = dfmaster2.drop("company_permalink")

### investigate quarter of founding and funds

In [135]:
#summarize raised amount of USD for each quarter
investperquarter = dfmaster3.\
select("quarter_new", "raised_amount_usd").\
groupby("quarter_new").\
agg(fn.sum("raised_amount_usd")).\
sort("quarter_new")

In [136]:
investperquarternona = investperquarter.na.drop()

In [137]:
#companies which were founded in quarter 1 tend to get more funds than others
display(investperquarternona)

### join custom category column to dataframe of companies and investments data

In [139]:
#select permalink, category and kmeans_feat
df_join1 = df_catid_kmeans.\
  select("permalink", "category", "kmeans_feat")

In [140]:
#join to final custom category df
df_final_permas = df_join1.join(final, df_join1["kmeans_feat"] == final["kmean_feat"], 'leftouter')

In [141]:
display(df_final_permas)

In [142]:
#select subset and rename permalink column to ensure unique column names
df_final_permas_sub = df_final_permas.\
  select(col("permalink").alias("permalink_sub"),"category_final")

In [143]:
#join custom categories to master df which is the one resulting from joining companies and investment spreadsheet (inlcuding aggregation)
df_master_final_cate = dfmaster_final.join(df_final_permas_sub, dfmaster_final["permalink"] == df_final_permas_sub["permalink_sub"], 'leftouter')

In [144]:
display(df_master_final_cate)

### get majority of investor_country_codes and funding_round_types to reduce complexity

Since a company could have had multiple investors from different countries and could have faced multiple different funding rounds, the aggregated data can contain a list of the different country codes of the investors or a list with different fuding round types. This leads to many different categories. To reduce dimension and complexity we decided to look for the most occurring investor country code or funding round type of each company and to use this value as representative label.

In [147]:
#select subset of appropriate columns
dfmastermajority = df_master_final_cate.select("permalink", "investor_country_codes", "funding_round_types")

In [148]:
#the blank strings have to get converted to NULL for further functions
dfmajority = dfmastermajority.\
  withColumn("investor_country_codes", blank_as_null("investor_country_codes")).\
  withColumn("funding_round_types", blank_as_null("funding_round_types"))

In [149]:
#write investor_country_codes string and funding_round_types strings that a separated by commas into a list
majority = dfmajority.\
  withColumn("investor_country_codes", split(col("investor_country_codes"), ",\s*")).\
  withColumn("funding_round_types", split(col("funding_round_types"), ",\s*"))

In [150]:
#dropping of rows with NULL values
majoritydropinv = majority.select("permalink","investor_country_codes").na.drop()
majoritydropround = majority.select("permalink","funding_round_types").na.drop()

In [151]:
# Counter vectorizing the investor_country_codes and funding_round_types feature in order to use the vector to calculate the majority count for each observation

cv_inv = feature.CountVectorizer(inputCol='investor_country_codes', outputCol='tf')

cv_round = feature.CountVectorizer(inputCol='funding_round_types', outputCol='tf')


cv_inv_model = cv_inv.fit(majoritydropinv)
df_cv_inv = cv_inv_model.transform(majoritydropinv)

cv_round_model = cv_round.fit(majoritydropround)
df_cv_round = cv_round_model.transform(majoritydropround)

In [152]:
#calculate the two highest counts of the investor_country_code for each company 
invtwo = two_highest_catpluscount(df_cv_inv)

In [153]:
#assign the words of the counter vectorizer of the investor_country_codes
inv_words = cv_inv_model.vocabulary

In [154]:
#assign the corresponding investor_country_codes to the counts for each company
invplus_words = add_inv_words(invtwo)

In [155]:
#calculate the two highest counts of the funding_round_types for each company 
roundtwo = two_highest_catpluscount(df_cv_round)

In [156]:
#assign the corresponding funding_round_types to the counts for each company
round_words = cv_round_model.vocabulary

In [157]:
#assign the corresponding funding_round_types to the counts for each company
roundpluswords = add_round_words(roundtwo)

In [158]:
#join the "majority" investor_country_codes to the master table
masternew = df_master_final_cate.join(invplus_words.select("perma",col("category").alias("investor_country_code")), df_master_final_cate["permalink"] == invplus_words["perma"], 'leftouter')

In [159]:
#selection in order to rename and for better joining
roundpluswords = roundpluswords.select(col("perma").alias("permaround"),col("category").alias("funding_round_type"))

In [160]:
#join the "majority" funding_round_types to the master table
masternew = masternew.join(roundpluswords, masternew["permalink"] == roundpluswords["permaround"], 'leftouter')

In [161]:
display(masternew)

In [162]:
#this dataframe was exported and then imported in another notebook 
#the entire project is split into two noteboos

masterdropped = masternew.drop("funding_total_usd", "permalink_agg", "investor_country_codes", "funding_round_types", "permalink_sub", "perma", "permaround")

In [163]:
masterdropped.count()

<p style="color: red">This dataframe called "masternew" is exported and then imported into Notebook 2.</p>

In [165]:
#display(masterdropped)

Some additional analysis (basic statistics) of the data in masternew follows here.

### create binary value for target variable

Since a classification predicts if a certain label case is true or not, the target variable needs to have a 1 or 0 representation.

In [169]:
#create new column with 1 or 0 depending on value of target variable: acquired = 1, otherwise 0
finaltarget = masterdropped.\
  withColumn("label", fn.when(col("status") == "acquired",1).otherwise(0))

In [170]:
display(finaltarget)

### drop missing values

In [172]:
#create subset of dataframe without missing values
finalwithoutna = finaltarget.na.drop()

# Understanding of final dataset without missing values

The final mastertable is also examined, since data preparations added new columns and missing values are dropped.

In [175]:
df = finalwithoutna

In [176]:
#distribution of categories in target variable
display(
  df.select("status").\
  groupby(col("status")).\
  agg(fn.count("status"))
)

In [177]:
#class balance target label
display(
  df.select("label").\
  groupby(col("label")).\
  agg(fn.count("label"))
)

In [178]:
#which categories occur most often
categories_master = df.select("category_final").\
  groupby(col("category_final")).\
  agg(fn.count("category_final")).\
  sort("count(category_final)", ascending=False)

In [179]:
display(categories_master.take(10))

In [180]:
#investigate in which cities most of the startups are
cities_master = df.select("name", "city").\
  groupBy("city").\
  agg(fn.count("city")).\
  sort("count(city)", ascending=False)

In [181]:
display(cities_master.take(10))

In [182]:
#investigate in which countries most of the startups of our data are
countries_master = df.select("name", "country_code").\
  groupBy("country_code").\
  agg(fn.count("country_code")).\
  sort("count(country_code)", ascending=False)

In [183]:
display(countries_master)

In [184]:
# get compnay name, number of investment and total amount of invested usd sorted by number of investors
investors_master_count = df.\
  select("name", "count_investor", "total_raised_usd", "category_final").\
  groupby(col("name")).\
  agg(fn.max("count_investor"), fn.max("total_raised_usd"), fn.first("category_final")).\
  sort("max(count_investor)", ascending=False)

In [185]:
display(investors_master_count.take(10))

In [186]:
# get investor name, number of investment and total amount of invested usd sorted by raised usd
investors_master_usd = df.\
  select("name", "count_investor", "total_raised_usd", "category_final").\
  groupby(col("name")).\
  agg(fn.max("count_investor"), fn.max("total_raised_usd"), fn.first("category_final")).\
  sort("max(total_raised_usd)", ascending=False)

In [187]:
display(investors_master_usd.take(10))

In [188]:
# get categories of funding rounds and the number of their occurrence
fundingrounds_master = df.\
  select("funding_round_type").\
  groupBy("funding_round_type").\
  agg(fn.count("funding_round_type")).\
  sort("count(funding_round_type)", ascending=False)

In [189]:
display(fundingrounds_master.take(5))

# End of notebook 1 #
We decided to split our notebook, because it became too long and we experienced some delay when scrolling or running code. The dataframe called "masternew" was exported, uploaded to Dropbox and will be imported at the very beginning of <strong>Notebook 2</strong>.